<a href="https://colab.research.google.com/github/Rohit-Munda/GenAIWorkshop/blob/main/RetrievalQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.9 MB/s eta 0:00:00


✅ Required Libraries

In [2]:
import faiss
import numpy as np
from transformers import pipeline
from sentence_transformers import SentenceTransformer

📚 1. Create a Small Knowledge Base (Documents)

In [4]:
documents = [
    "The Earth revolves around the Sun in about 365 days.",
    "Python is a popular programming language used for web development, data science, and AI.",
    "The capital of France is Paris, known for the Eiffel Tower.",
    "Photosynthesis is the process by which green plants make food using sunlight.",
    "Basketball is a sport played by two teams of five players on a rectangular court.",
    "Large Language Models like ChatGPT are trained on vast amounts of text data."
]

🔡 2. Convert Documents to Embeddings

In [5]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(documents)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
doc_embeddings

array([[ 0.02731097,  0.0774596 ,  0.04154115, ..., -0.00238924,
         0.00900925, -0.0208409 ],
       [-0.04744018, -0.01688633, -0.02255926, ...,  0.12869716,
         0.1589973 ,  0.0186499 ],
       [ 0.0672824 ,  0.06106832,  0.02706188, ...,  0.06351971,
         0.11277966,  0.04227098],
       [-0.05481903,  0.06031541, -0.06253127, ...,  0.05017699,
         0.11442049,  0.03344167],
       [ 0.06275784,  0.01622444,  0.03321232, ...,  0.07344805,
         0.06807995, -0.00025563],
       [-0.01688357, -0.08255044,  0.06435584, ...,  0.05519102,
         0.03942716, -0.04331229]], dtype=float32)

🧠 3. Create FAISS Index for Retrieval

In [7]:
embedding_dim = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(doc_embeddings))

🔍 4. Define a Retrieval + QA Function (Mini-RAG)

In [8]:
# Use a model for QA
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


In [9]:
def rag_answer(question, top_k=2):
    # Embed the question
    question_embedding = embedder.encode([question])

    # Retrieve top_k similar documents
    distances, indices = index.search(np.array(question_embedding), top_k)
    retrieved_docs = [documents[i] for i in indices[0]]

    # Combine docs into a single context string
    context = " ".join(retrieved_docs)

    # Run QA
    result = qa_pipeline(question=question, context=context)

    # Output
    print(f"📌 Question: {question}")
    print(f"\n📚 Retrieved Context:\n{context}")
    print(f"\n🧠 Answer: {result['answer']}")

In [10]:
rag_answer("What is photosynthesis?")

📌 Question: What is photosynthesis?

📚 Retrieved Context:
Photosynthesis is the process by which green plants make food using sunlight. The Earth revolves around the Sun in about 365 days.

🧠 Answer: the process by which green plants make food using sunlight


In [12]:
rag_answer("Where is Eiffel Tower located?")

📌 Question: Where is Eiffel Tower located?

📚 Retrieved Context:
The capital of France is Paris, known for the Eiffel Tower. The Earth revolves around the Sun in about 365 days.

🧠 Answer: Paris


In [13]:
rag_answer("What is Python used for?")

📌 Question: What is Python used for?

📚 Retrieved Context:
Python is a popular programming language used for web development, data science, and AI. Large Language Models like ChatGPT are trained on vast amounts of text data.

🧠 Answer: web development, data science, and AI


# ✅ What We Did: A Simple RAG Pipeline

- Embedded documents into vectors
- Used FAISS to retrieve relevant documents based on question
- Passed retrieved text + question into a question-answering model
- Got grounded answers instead of hallucinations!